## Prepare Scout Data for both RECS and SEDS analysis

In [ ]:
## 1. Data preparation for Scout output using Stock data (non-electricity)
import json
import matplotlib.pyplot as plt
import pandas as pd
# Load the JSON files

file_dir = "test_compare_base_disagg"
recs_or_seds = "seds"
# recs_or_seds = "recs"

eussversion = "2024"
# eussversion = "2023"

if recs_or_seds == "recs":
    energy_or_stock = "stock"
    fueltype = "electricity"
    if eussversion == "2024":
        file_paths = [f"./{file_dir}/jaredstest/mseg_res_com_state_2024_tech_factors.json", f"./{file_dir}/2023_work/mseg_res_com_state_2023.json"]
    elif eussversion == "2023":
        file_paths = [f"./{file_dir}/2023_work/mseg_res_com_state_2023.json", f"./{file_dir}/2023_work/mseg_res_com_state_2023.json"]
elif recs_or_seds == "seds":
    # projectionyear = "2023"
    projectionyear = "2016"
    
    fueltype = "electricity"
    # fueltype = 'natural gas'
    
    energy_or_stock = "energy"

    if eussversion == "2024":
        if fueltype == "electricity":
            file_paths = [f"./{file_dir}/jaredstest/mseg_res_com_state_2024_tech_factors.json", f"./{file_dir}/2023_work/mseg_res_com_state_2023.json"]
        elif fueltype == "natural gas":
            file_paths = [f"./{file_dir}/jaredstest/mseg_res_com_state_2024_eu_factors.json", f"./{file_dir}/2023_work/mseg_res_com_state_2023.json"]
    elif eussversion == "2023":
        file_paths = [f"./{file_dir}/2023_work/mseg_res_com_state_2023.json", f"./{file_dir}/2023_work/mseg_res_com_state_2023.json"]

In [ ]:
datasets = []

for path in file_paths:
    with open(path, "r") as file:
        datasets.append(json.load(file))

states = set(datasets[0].keys()).intersection(set(datasets[1].keys()))

In [ ]:



def get_overall_data(fueltype, energy_or_stock, rettype):

    end_uses_all = set()
    for state in states:
        for building_type in datasets[0][state]:
            for energy_source in datasets[0][state][building_type]: 
                if energy_source == fueltype:
                    end_uses_all.update(datasets[0][state][building_type][energy_source].keys())
    
    res_bldgs = ['single family home', 'multi family home', 'mobile home']
    end_uses_all = list(end_uses_all) 
    end_uses_wsupply = ["heating", "cooling", "secondary heating"]
    years = [str(year) for year in range(2015, 2051)]
    
    allbldgs_data = [{state: {end_use: {year: 0 for year in years} for end_use in end_uses_all} for state in states} for _ in range(2)]
    resbldgs_data = [{state: {end_use: {year: 0 for year in years} for end_use in end_uses_all} for state in states} for _ in range(2)]
    combldgs_data = [{state: {end_use: {year: 0 for year in years} for end_use in end_uses_all} for state in states} for _ in range(2)]
    
    def is_valid_stock(stock):
        # If stock is a string and equals "NA", it's not valid.
        if isinstance(stock, str):
            if stock == "NA":
                return False
            # Otherwise, if it's a non-"NA" string, you could choose to accept or reject it.
            return True
    
        # If stock is a dictionary, check its year values.
        elif isinstance(stock, dict):
            # If the dictionary is empty, it isn't valid.
            if not stock:
                return False
            # If all values in the dictionary are 0.0, then it's not valid.
            if all(float(value) == 0.0 for value in stock.values()):
                return False
            # Otherwise, it has at least one non-zero value.
            return True
    
        # Any other type is considered invalid.
        return False
    
    for dataset_index, dataset in enumerate(datasets):
        for state in states:
            for building_type in dataset [state]:
                for energy_source in dataset [state][building_type]:
                    if energy_source == fueltype:
                        for end_use in dataset [state][building_type][energy_source]:
                            if end_use in end_uses_wsupply:
                                for subcategory in dataset [state][building_type][energy_source][end_use]['supply']:
                                    if is_valid_stock(dataset [state][building_type][energy_source][end_use]['supply'][subcategory][energy_or_stock]):
                                        for year, value in dataset [state][building_type][energy_source][end_use]['supply'][subcategory][energy_or_stock].items():
                                            if year in allbldgs_data[dataset_index][state][end_use]:
                                                allbldgs_data[dataset_index][state][end_use][year] += value
                                            if building_type in res_bldgs:
                                                if year in resbldgs_data[dataset_index][state][end_use]:
                                                    resbldgs_data[dataset_index][state][end_use][year] += value
                                            if building_type not in res_bldgs:
                                                if year in combldgs_data[dataset_index][state][end_use]:
                                                    combldgs_data[dataset_index][state][end_use][year] += value
                            if end_use not in end_uses_wsupply:
                                for subcategory in dataset [state][building_type][energy_source][end_use]:
                                    if subcategory != 'energy' and subcategory != 'stock':
                                        if is_valid_stock(dataset [state][building_type][energy_source][end_use][subcategory][energy_or_stock]):
                                            for year, value in dataset [state][building_type][energy_source][end_use][subcategory][energy_or_stock].items():
                                                if year in allbldgs_data[dataset_index][state][end_use]:  # Ensure year is valid
                                                    allbldgs_data[dataset_index][state][end_use][year] += value
                                                if building_type in res_bldgs:
                                                    if year in resbldgs_data[dataset_index][state][end_use]:
                                                        resbldgs_data[dataset_index][state][end_use][year] += value
                                                if building_type not in res_bldgs:
                                                    if year in combldgs_data[dataset_index][state][end_use]:
                                                        combldgs_data[dataset_index][state][end_use][year] += value
                                    else:
                                        if is_valid_stock(dataset [state][building_type][energy_source][end_use][energy_or_stock]):
                                            for year, value in dataset [state][building_type][energy_source][end_use][energy_or_stock].items():
                                                if year in allbldgs_data[dataset_index][state][end_use]:  # Ensure year is valid
                                                    allbldgs_data[dataset_index][state][end_use][year] += value
                                                if building_type in res_bldgs:
                                                    if year in resbldgs_data[dataset_index][state][end_use]:
                                                        resbldgs_data[dataset_index][state][end_use][year] += value
                                                if building_type not in res_bldgs:
                                                    if year in combldgs_data[dataset_index][state][end_use]:
                                                        combldgs_data[dataset_index][state][end_use][year] += value
    print(f"{fueltype}: COMBINE ALL SECTORS")
    allbldgs_dfs = [
        {state: pd.DataFrame.from_dict(allbldgs_data[dataset_index][state], orient="index", columns=years).T for state in states}
        for dataset_index in range(2)
    ]
    
    print(f"{fueltype}: RESIDENTIAL SECTOR")
    resbldgs_dfs = [
        {state: pd.DataFrame.from_dict(resbldgs_data[dataset_index][state], orient="index", columns=years).T for state in states}
        for dataset_index in range(2)
    ]
    print(f"{fueltype}: COMMERCIAL SECTOR")
    combldgs_dfs = [
        {state: pd.DataFrame.from_dict(combldgs_data[dataset_index][state], orient="index", columns=years).T for state in states}
        for dataset_index in range(2)
    ]
    if rettype:
        return resbldgs_data, combldgs_data
    else:
        return resbldgs_dfs, combldgs_dfs

scout_res_elec_eu, scout_com_elec_eu = get_overall_data("electricity", "energy", True)
scout_res_ng_eu, scout_com_ng_eu = get_overall_data("natural gas", "energy", True)

## Validate with SEDS Data

In [ ]:
# Prepare Scout results
def convert_to_df_for_seds(data, yr, title):
    # Initialize lists to store the data
    rows = []
    
    # Process the nested dictionary
    for state, end_uses in data.items():  # Directly iterate over the dictionary
        for end_use, years in end_uses.items():
            for year, value in years.items():
                row = {
                    'state': state,
                    'year': year,
                    'end_use': end_use,
                    'value': value
                }
                rows.append(row)
    
    # Create DataFrame in long format
    df = pd.DataFrame(rows)
    df = df[df['year'] == yr]
    
    # Check for duplicates
    duplicates = df.duplicated(subset=['state', 'year', 'end_use'], keep=False)
    if duplicates.any():
    
        # Add a count column to handle duplicates
        df['count'] = df.groupby(['state', 'year', 'end_use']).cumcount()
        # Create unique end_use names by adding count suffix for duplicates
        df['end_use_unique'] = df.apply(lambda x: f"{x['end_use']}_{x['count']}" if x['count'] > 0 else x['end_use'], axis=1)
        
        # Pivot to wide format using the unique end_use names
        df_pivot = df.pivot(index=['state', 'year'], 
                           columns='end_use_unique', 
                           values='value').reset_index()
    else:
        # print("No duplicates found, converting to wide format directly")
        # If no duplicates, use regular pivot
        df_pivot = df.pivot(index=['state', 'year'], 
                           columns='end_use', 
                           values='value').reset_index()
    
    numeric_columns = [col for col in df_pivot.columns if col not in ['state', 'year']]
    df_pivot[title] = df_pivot[numeric_columns].sum(axis=1)
    df_pivot = df_pivot[['state', title]]
    # print(df_pivot.to_string(index=False))
    return df_pivot


scout_res_elec = convert_to_df_for_seds(scout_res_elec_eu[0],projectionyear, 'scout_res_elec')
scout_com_elec = convert_to_df_for_seds(scout_com_elec_eu[0], projectionyear, 'scout_com_elec')
scout_res_ng = convert_to_df_for_seds(scout_res_ng_eu[0],projectionyear, 'scout_res_ng')
scout_com_ng = convert_to_df_for_seds(scout_com_ng_eu[0], projectionyear, 'scout_com_ng')

scout_dat = pd.merge(scout_res_elec, scout_com_elec, on='state', how='inner')
scout_dat = pd.merge(scout_dat, scout_res_ng, on='state', how='inner')
scout_dat = pd.merge(scout_dat, scout_com_ng, on='state', how='inner')
# print(scout_dat.to_string(index=False))

elec_factor = 0.29307107
ng_factor = 1e6 # 1,000,000

# Apply multiplications to the respective columns
scout_dat['scout_res_elec_mwh'] = scout_dat['scout_res_elec'] * elec_factor
scout_dat['scout_com_elec_mwh'] = scout_dat['scout_com_elec'] * elec_factor
scout_dat['scout_res_ng_tbtu'] = scout_dat['scout_res_ng'] / ng_factor
scout_dat['scout_com_ng_tbtu'] = scout_dat['scout_com_ng'] / ng_factor
scout_dat.to_csv(f"./{file_dir}/scout_dat_euss{eussversion}_proj{projectionyear}.csv", index=False)

In [ ]:
import pandas as pd
import os
# https://www.eia.gov/state/seds/data.cfm?incfile=/state/seds/sep_fuel/html/fuel_use_ng.html&sid=US&sid=CA
# https://www.eia.gov/electricity/data/state/xls/861/HS861%202010-.xlsx

seds_dir = "seds"
if projectionyear == "2016":
    seds_file = "seds2016.csv"
elif projectionyear == "2023":
    seds_file = "seds2023.csv"
seds_file_path = os.path.join(seds_dir, seds_file)
seds_dat = pd.read_csv(seds_file_path)

scout_seds_dat = pd.merge(scout_dat, seds_dat, on='state', how='inner')
scout_seds_dat = scout_seds_dat[~scout_seds_dat['state'].isin(['AK', 'HI'])]
# scout_seds_dat.to_csv("scout_seds_dat.csv", index=False)
# print(scout_seds_dat)


In [ ]:
df = scout_seds_dat
file_dir = "test_compare_base_disagg"
pairs = [
    ('scout_res_elec_mwh', 'seds_res_elec_mwh', 'Residential Electricity'),
    ('scout_com_elec_mwh', 'seds_com_elec_mwh', 'Commercial Electricity'),
    ('scout_res_ng_tbtu', 'seds_res_ng_tbtu', 'Residential Natural Gas'),
    ('scout_com_ng_tbtu', 'seds_com_ng_tbtu', 'Commercial Natural Gas')
]

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.flatten()  # Flatten the 2x2 array for easier iteration

# Plot each pair in the grid
for idx, (x_col, y_col, title_prefix) in enumerate(pairs):
    if "Electricity" in title_prefix:
        lbl_unit = 'MWh'
    else:
        lbl_unit = 'TBtu'
    
    ax = axes[idx]
    
    # Scatter plot
    ax.scatter(df[x_col], df[y_col], color='blue', label='States')
    
    # Add state labels
    for i, state in enumerate(df['state']):
        ax.text(df[x_col].iloc[i], df[y_col].iloc[i], state, fontsize=8, ha='right')
    
    # Plot y=x line
    max_val = max(df[x_col].max(), df[y_col].max())
    min_val = min(df[x_col].min(), df[y_col].min())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='y = x')
    
    # Set labels and title
    ax.set_xlabel(f"Scout ({lbl_unit})", fontsize=10)
    ax.set_ylabel(f"SEDS ({lbl_unit})", fontsize=10)
    ax.set_title(f'Scout and SEDS ({title_prefix})\nEUSS={eussversion},Proj.={projectionyear}', fontsize=12)
    
    # Set scientific notation for axes
    ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))
    
    # Add legend
    ax.legend()
    
    # Add grid
    ax.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.savefig(f"./{file_dir}/scout_seds_EUSS{eussversion}_{fueltype}{projectionyear}.png")
plt.show()

## Validate with RECS

### Prepare Scout

In [ ]:
def merge_wscoutgeo_cdiv_calc(df):
    scoutgeo = f"new_scout_geography.csv"
    scoutgeo_df = pd.read_csv(scoutgeo)
    new_scoutgeo_df = scoutgeo_df[['state', 'cdiv', 'cdiv_high']]
    new_scoutgeo_df = new_scoutgeo_df.drop_duplicates()    

    final_df = df[df['Year'] == '2020']
    
    final_df = final_df.merge(new_scoutgeo_df, left_on="State", right_on="state", how="left")
    final_df.drop(columns=["State"], inplace=True, errors='ignore')


    return final_df
    
def merge_dfs(dfs):
    merged_dfs = []
    
    for dataset_index, dataset in enumerate(dfs):
        if dataset_index == 0:
            for state, df in dataset.items():
                df = df.copy()
                df['State'] = state
                # df['Dataset_Index'] = dataset_index  # Track dataset index
                merged_dfs.append(df.reset_index().rename(columns={'index': 'Year'}))
    
    # Concatenating all dataframes into one
    final_df = pd.concat(merged_dfs, ignore_index=True)
    final_df2 = merge_wscoutgeo_cdiv_calc(final_df)

    columns_of_interest = ["heating", "cooking", "water heating"]
    df_totals = final_df2.groupby("cdiv")[columns_of_interest].transform("sum")
    
    for col in columns_of_interest:
        final_df2[f"{col}_ratio"] = final_df2[col] / df_totals[col]
    
    final_df2 = final_df2[['state']+[s + "_ratio" for s in columns_of_interest]]
    
    return final_df2

dir_out = "test_compare_base_disagg"
scout_res_elec, scout_com_elec = get_overall_data("electricity", energy_or_stock, False)

merge_dfs(scout_res_elec).to_csv(f"./{dir_out}/{energy_or_stock}_{eussversion}_res.csv", index=False)

### Prepare RECS

In [ ]:
## Prepare RECS data 
import pandas as pd
import numpy as np
def recs_merge_wscoutgeo_cdiv_calc(df):
    scoutgeo = f"new_scout_geography.csv"
    scoutgeo_df = pd.read_csv(scoutgeo)
    new_scoutgeo_df = scoutgeo_df[['state', 'cdiv', 'cdiv_high']]
    new_scoutgeo_df = new_scoutgeo_df.drop_duplicates()    

    final_df = df
    
    final_df = final_df.merge(new_scoutgeo_df, left_on="State", right_on="state", how="left")
    final_df.drop(columns=["State"], inplace=True, errors='ignore')
    return final_df

def recs_calc_ratio(data, columns_of_interest):
    final_df = data.copy()
    final_df = recs_merge_wscoutgeo_cdiv_calc(final_df)

    # Replace strings with NaN across all columns of interest
    final_df[columns_of_interest] = final_df[columns_of_interest].apply(
        lambda x: pd.to_numeric(x, errors='coerce')
    )
    
    # Calculate totals per group, skipping NaN values
    df_totals = final_df.groupby("cdiv")[columns_of_interest].transform(
        lambda x: x.sum(skipna=True)
    )

    for col in columns_of_interest:
        final_df[f"{col}_ratio"] = np.where(
            final_df[col].notnull() & df_totals[col].notnull(),
            final_df[col] / df_totals[col],
            'NA'
        )
    
    final_df = final_df[['state']+[s + "_ratio" for s in columns_of_interest]]
    return final_df

dir_name = "recs_docs"
dir_out = "test_compare_base_disagg"

data = pd.read_csv(f"{dir_name}/state_appliances.csv")
appliances_df = recs_calc_ratio(data, ["Electric cooking appliance (Number)"])
# appliances_df.to_csv(f"{dir_out}/recs_appliances.csv",index=False)

data = pd.read_csv(f"{dir_name}/state_space_heating.csv")
heating_df = recs_calc_ratio(data, ["Central heat pump (Number)"])
# heating_df.to_csv(f"{dir_out}/recs_heating.csv",index=False)

data = pd.read_csv(f"{dir_name}/state_water_heating.csv")
water_heating_df = recs_calc_ratio(data, ["Electricity (Number)"])
# water_heating_df.to_csv(f"{dir_out}/recs_water_heating.csv",index=False)

water_heating_df = water_heating_df.rename(columns={"Electricity (Number)_ratio": "recs_water_heating_ratio"})
heating_df = heating_df.rename(columns={"Central heat pump (Number)_ratio": "recs_heating_ratio"})
appliances_df = appliances_df.rename(columns={"Electric cooking appliance (Number)_ratio": "recs_cooking_ratio"})

# Merge the DataFrames on 'state' using outer join
recs_df = pd.merge(water_heating_df, heating_df, on="state", how="outer")
recs_df = pd.merge(recs_df, appliances_df, on="state", how="outer")
recs_df.to_csv(f"{dir_out}/recs_data.csv",index=False)


### Analyze with RECS

In [ ]:
# ANALYSIS with RECS
###########################################################################################



###########################
recs_df = pd.read_csv(f"{dir_out}/recs_data.csv")
stock_df = pd.read_csv(f"{dir_out}/{energy_or_stock}_{eussversion}_res.csv") # Got the data from running 1. Data preparation for Scout output using Stock data
stock_recs_df = pd.merge(stock_df, recs_df, on="state", how="outer")
stock_recs_df.to_csv(f"{dir_out}/stock_recs.csv",index=False)
###########################

df = stock_recs_df.copy()

# Define the pairs of columns to compare
comparisons = [
    ("heating_ratio", "recs_heating_ratio", "Heating (Year 2020)"),
    ("cooking_ratio", "recs_cooking_ratio", "Cooking (Year 2020)"),
    ("water heating_ratio", "recs_water_heating_ratio", "Water Heating (Year 2020)")
]

# Create a 1x3 grid of subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)

# Plot each comparison
for idx, (x_col, y_col, title) in enumerate(comparisons):
    ax = axes[idx]
    
    # Scatter plot for states (drop rows where either x or y is NaN)
    plot_df = df.dropna(subset=[x_col, y_col])
    ax.scatter(plot_df[x_col], plot_df[y_col], color='blue', label='States')
    
    # Add state labels to each point
    for i, state in enumerate(plot_df['state']):
        ax.annotate(state, (plot_df[x_col].iloc[i], plot_df[y_col].iloc[i]), fontsize=8)
    
    # Plot y=x line
    ax.plot([0, 1], [0, 1], 'r--', label='y = x')
    
    # Set labels and title
    ax.set_xlabel(f"Using tech factors: {title.split()[0]}")
    ax.set_ylabel(f"Using RECS: {title.split()[0]}")
    ax.set_title(title)
    
    # Set log scale (optional, based on the image; adjust limits as needed)
    ax.set_xscale('linear')
    ax.set_yscale('linear')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    # Add legend
    ax.legend()

# Adjust layout to prevent overlap
plt.tight_layout()
# plt.show()
plt.savefig(f"./{dir_out}/scout_recs_version_{eussversion}.png")